The following script accepts six projects with a control group (all phenotypes are healthy) in order to examine the differences between the projects and create a homogeneous group.

In [2]:
#upload packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from Summary_Project import analysis, results
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
import warnings
import time

In [3]:
warnings.filterwarnings("ignore")

In [4]:
IBD_1000 = results["IBD_1000"]
IBD_1000_T = results["IBD_1000_T"]

Eran_Elinav = results['Eran_Elinav']
Eran_Elinav_T = results['Eran_Elinav_T']

Gevers = results['Gevers']
#Gevers_T = results['Gevers_T']

HMP1 = results['HMP1']
HMP1_T = results['HMP1_T']

HMP2 = results['HMP2']
HMP2_T = results['HMP2_T']

HMP2_pilot = results['HMP2_pilot']
HMP2_pilot_T = results['HMP2_pilot_T']

HMP3 = results['HMP3']
HMP3_T = results['HMP3_T']

ibdmdb = results['ibdmdb']
ibdmdb_T = results['ibdmdb_T']

ibdmdb2 = results['ibdmdb2']
ibdmdb2_T = results['ibdmdb2_T']

Lewis = results['Lewis']
Lewis_T = results['Lewis_T']

MetaHit = results['MetaHit']
MetaHit_T = results['MetaHit_T']

MetaHit2 = results['MetaHit2']
MetaHit2_T = results['MetaHit2_T']

disease_counts = results['disease_counts']
disease_counts

,IBD_1000,EranElinav,Gevers,HMP1,HMP2,HMP3,IBDMDB,HMP2_pilot,IBDMDB2,Lewis,MetaHit,MetaHit2,Total_Patients
Diagnosis,,,,,,,,,,,,,
CD,206,36,36,0,0,0,62,59,50,87,4,13,553
UC,126,24,4,0,0,0,35,34,30,0,21,69,343
IBD,23,0,1,0,0,0,0,0,0,0,0,0,24
nonIBD,0,22,9,140,93,89,24,24,26,26,99,189,741


Importing the metadata and the metaphlan4 files that show the phenotype and the species abundances from the samples.

The cells that follow randomly sample 20 samples from each project and keep only the species in each metaphlan file.

In [198]:
HMP1_METADATA = HMP1_METADATA.sample(20)
HMP1_metaphlan4 = HMP1_metaphlan4.loc[HMP1_METADATA.index,:]
HMP1_metaphlan4 = HMP1_metaphlan4.loc[:,HMP1_metaphlan4.columns.str.startswith('s__')]
HMP1_metaphlan4["Type"] = 'HMP1'
HMP1_metaphlan4

Index(['SAMN00045227', 'SAMN00083009', 'SAMN00037274', 'SAMN00040800',
       'SAMN00036649', 'SAMN00039351', 'SAMN00034694', 'SAMN00088343',
       'SAMN00041005', 'SAMN00037971',
       ...
       'SAMN00037313', 'SAMN00063710', 'SAMN00044801', 'SAMN00036881',
       'SAMN00040615', 'SAMN00062637', 'SAMN00040202', 'SAMN00040379',
       'SAMN00037421', 'SAMN00040485'],
      dtype='object', length=140)

In [203]:
HMP2_METADATA = HMP2_METADATA.sample(20)
HMP2_metaphlan4 = HMP2_metaphlan4.loc[HMP2_METADATA.index,:]
HMP2_metaphlan4 = HMP2_metaphlan4.loc[:,HMP2_metaphlan4.columns.str.startswith('s__')]
HMP2_metaphlan4["Type"] = 'HMP2'
HMP2_metaphlan4

94

In [205]:
HMP3_METADATA = HMP3_METADATA[HMP3_METADATA['isolation_source'] == 'GUT']
HMP3_METADATA = HMP3_METADATA.sample(20)
HMP3_metaphlan4 = HMP3_metaphlan4.loc[HMP3_METADATA.index,:]
HMP3_metaphlan4 = HMP3_metaphlan4.loc[:,HMP3_metaphlan4.columns.str.startswith('s__')]
HMP3_metaphlan4["Type"] = 'HMP3'
HMP3_metaphlan4

89

In [160]:
select_sample_L = Lewis_METADATA.groupby(['Subject']).agg({'FCP': max,'sample_accession':'first'})
select_sample_L.set_index('sample_accession', inplace=True)
print(f'Number of unique patients: {len(Lewis_METADATA["Subject"].unique())}')
# control_group = select_sample_L.iloc[0:26,:]
# Lewis_control_METADATA = control_group.sample(20)
# Lewis_metaphlan4 = Lewis_metaphlan4.loc[Lewis_control_METADATA.index,:]
# Lewis_metaphlan4 = Lewis_metaphlan4.loc[:,Lewis_metaphlan4.columns.str.startswith('s__')]
# Lewis_metaphlan4["Type"] = 'Lewis'
# Lewis_metaphlan4

Number of unique patients: 116


Abundance,s__Roseburia_faecis,s__Ruminococcus_bromii,s__Blautia_wexlerae,s__Flavonifractor_plautii,s__Ruminococcus_gnavus,s__Blautia_obeum,s__Bifidobacterium_breve,s__Roseburia_inulinivorans,s__Bifidobacterium_longum,s__Tyzzerella_nexilis,...,s__GGB4739_SGB6564,s__Candidatus_Heteroscilispira_lomanii,s__Anaerofilum_sp_An201,s__GGB28645_SGB41267,s__Phocaeicola_sartorii,s__Lactobacillus_acetotolerans,s__Secundilactobacillus_silagei,s__GGB2722_SGB3663,s__Paracoccus_yeei,Type
sample_accession,,,,,,,,,,,,,,,,,,,,,
SAMN03469246,0.00255,0.00263,0.18157,0.00000,0.00000,0.17281,0.00000,0.02512,0.01173,0.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lewis
SAMN03469239,0.00000,0.00000,0.17556,0.20604,0.16684,0.01956,0.00000,0.00000,0.01824,0.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lewis
SAMN03469242,1.37296,1.66555,0.38425,0.46216,0.21874,0.05520,0.00000,0.00000,0.45782,0.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lewis
SAMN03469231,0.08474,3.18150,0.39988,0.00000,0.00000,0.00124,0.00000,0.24112,0.20543,0.09601,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lewis
SAMN03469238,0.63669,0.00000,0.54396,0.36239,0.11360,0.00000,0.00000,0.00000,0.00000,0.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lewis
SAMN03469228,0.06484,0.26504,0.01105,0.00000,0.05554,0.01900,0.00000,0.00000,0.03671,0.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lewis
SAMN03469225,0.00000,2.80496,5.32578,0.85846,0.10032,0.02975,0.01246,0.04688,0.54181,0.02521,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lewis
SAMN03469244,0.00000,21.13649,0.00000,0.00000,0.00000,0.11381,0.00000,0.00000,0.00000,0.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lewis
SAMN03469234,2.15089,0.00000,0.68265,0.00000,0.00000,0.02730,0.00000,0.65641,0.09142,0.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lewis


In [161]:
IBD_METADATA = IBD_METADATA[IBD_METADATA['Diagnosis'] == 'nonIBD']
IBD_METADATA = IBD_METADATA.sample(20)
IBD_metaphlan4 = IBD_metaphlan4.loc[IBD_METADATA.index,:]
IBD_metaphlan4 = IBD_metaphlan4.loc[:,IBD_metaphlan4.columns.str.startswith('s__')]
IBD_metaphlan4["Type"] = 'IBDMDB'
IBD_metaphlan4

Abundance,s__Phocaeicola_vulgatus,s__Phocaeicola_massiliensis,s__Prevotella_copri_clade_A,s__Alistipes_putredinis,s__Faecalibacterium_prausnitzii,s__Barnesiella_intestinihominis,s__Akkermansia_muciniphila,s__Sutterella_wadsworthensis,s__Parabacteroides_merdae,s__Bacteroides_caccae,...,s__GGB35456_SGB47582,s__GGB3171_SGB4185,s__GGB42689_SGB59892,s__Slackia_piriformis,s__Schaalia_turicensis,s__Bacteroidales_bacterium_KA00251,s__Parvimonas_sp_KA00067,s__Urmitella_timonensis,s__GGB1455_SGB2018,Type
External ID,,,,,,,,,,,,,,,,,,,,,
MSM79H58,2.26628,0.00000,0.00000,2.20974,0.33228,2.39906,2.09257,0.00000,0.00000,1.47448,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IBDMDB
MSM6J2HT,27.07307,0.00000,0.00000,7.20628,0.43142,2.79273,0.00000,0.00000,1.63631,4.54129,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IBDMDB
MSM9VZFR,52.72668,0.00000,0.00000,2.90623,3.19605,0.85822,0.00000,0.76437,1.28386,2.39752,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IBDMDB
MSM6J2K6,6.01520,3.57281,50.17500,1.76351,0.97157,2.27390,0.00000,0.50301,0.00000,1.47527,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IBDMDB
MSM79H8D,0.00000,3.95866,0.00000,5.81480,3.41456,1.85760,0.00000,1.91419,3.42323,0.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IBDMDB
MSM6J2JR,14.65194,0.00000,0.00000,0.00000,0.53095,0.00000,22.88893,0.00000,2.69239,24.29683,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IBDMDB
HSM5MD5D,13.92740,0.00000,0.00000,0.00000,15.49314,0.00000,0.00000,0.00000,3.70612,2.41710,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IBDMDB
HSM67VGA,17.09180,0.00000,1.32641,4.68349,9.03756,0.00000,4.83911,0.00000,0.00000,3.76056,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IBDMDB
HSMA33NY,26.18488,0.00000,16.32606,3.19819,11.36687,0.51892,0.00000,1.72120,2.00958,0.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IBDMDB


In [212]:
MetaHit_METADATA = MetaHit_METADATA[MetaHit_METADATA['IBD'] == 'N']
MetaHit_METADATA = MetaHit_METADATA.sample(20)
MetaHit_metaphlan4 = MetaHit_metaphlan4.loc[MetaHit_METADATA.index,:]
MetaHit_metaphlan4 = MetaHit_metaphlan4.loc[:,MetaHit_metaphlan4.columns.str.startswith('s__')]
MetaHit_metaphlan4["Type"] = 'MetaHit'
MetaHit_metaphlan4

Abundance,s__Phocaeicola_plebeius,s__Ruminococcus_bicirculans,s__Eubacterium_rectale,s__Bacteroides_uniformis,s__Roseburia_faecis,s__Clostridia_unclassified_SGB4367,s__Ruminococcus_bromii,s__Phocaeicola_dorei,s__Alistipes_putredinis,s__Faecalibacterium_prausnitzii,...,s__GGB13083_SGB20273,s__GGB9686_SGB15190,s__Lelliottia_amnigena,s__Raoultella_ornithinolytica,s__Kosakonia_cowanii,s__Klebsiella_oxytoca,s__GGB14014_SGB21441,s__Enterobacter_huaxiensis,s__Sutterella_megalosphaeroides,Type
Sample,,,,,,,,,,,,,,,,,,,,,
SAMEA728884,0.00000,0.96148,0.99483,2.65988,4.76891,0.00000,2.83651,1.70301,0.08429,0.00000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MetaHit
SAMEA728856,7.84554,0.00000,5.43321,7.22744,0.00000,0.00000,0.00000,0.00093,0.00000,6.22272,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MetaHit
SAMEA728913,0.00000,0.11937,2.71508,5.47835,5.42712,0.00000,1.29290,0.00000,4.29094,2.83358,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MetaHit
SAMEA728614,0.02872,0.06823,1.91430,2.04070,1.68954,0.10086,0.00000,3.15181,2.49989,1.64470,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MetaHit
SAMEA728630,4.98585,0.08065,0.14800,3.27073,0.00000,0.00000,0.00000,0.38382,1.23760,3.12972,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MetaHit
SAMEA728674,0.00000,0.00000,2.56297,0.08386,0.16922,3.26584,1.88076,0.02060,0.00000,2.23349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MetaHit
SAMEA728714,0.01912,0.06464,2.80808,0.63683,2.62526,0.00000,3.05141,0.53131,1.15069,1.95236,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MetaHit
SAMEA728814,1.65429,0.00737,6.84330,1.18696,0.21758,0.00000,0.00000,0.66979,2.25157,2.29872,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MetaHit
SAMEA728697,0.49001,0.02411,1.86928,0.18250,0.42530,0.00000,0.00000,0.00000,1.18354,1.67989,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MetaHit


In [8]:
# 35 Metaphlan control samples are missing
ID_Eran_Elinav_C = Eran_Elinav.loc[Eran_Elinav['Disease'] == 'nonIBD', 'index'].values
Eran_Elinav_C = Eran_Elinav_T.loc[Eran_Elinav_T.index.intersection(ID_Eran_Elinav_C)]

# ID_Gevers_C = Gevers.loc[Gevers["Diagnosis"] == "nonIBD", ...].values - is missing  

ID_HMP1_C = HMP1.loc[HMP1["study design"] == 'nonIBD', 'index'].values
HMP1_C = HMP1_T.loc[HMP1_T.index.intersection(ID_HMP1_C)]

ID_HMP2_C = HMP2.loc[HMP1["study design"] == 'nonIBD', 'index'].values
HMP2_C = HMP2_T.loc[HMP2_T.index.intersection(ID_HMP2_C)]


ID_HMP2_pilot_C = HMP2_pilot.loc[HMP2_pilot["diagnosis"] == 'nonIBD', 'index'].values
HMP2_pilot_C = HMP2_pilot_T.loc[HMP2_pilot_T.index.intersection(ID_HMP2_pilot_C)]

ID_HMP3_C = HMP3.loc[HMP3["isolation_source"] == 'GUT', 'index'].values
HMP3_C = HMP3_T.loc[HMP3_T.index.intersection(ID_HMP3_C)]

# 2 Metaphlan control samples are missing
ID_ibdmdb_C = ibdmdb.loc[ibdmdb["diagnosis"] == 'nonIBD', 'index'].values
ibdmdb_C = ibdmdb_T.loc[ibdmdb_T.index.intersection(ID_ibdmdb_C)]

# 1 Metaphlan control samples are missing
ID_ibdmdb2_C = ibdmdb2.loc[ibdmdb2["diagnosis"] == 'nonIBD', 'index'].values
ibdmdb2_C = ibdmdb2_T.loc[ibdmdb2_T.index.intersection(ID_ibdmdb2_C)]

ID_Lewis_C = Lewis.loc[Lewis["diagnosis"] == 'nonIBD', 'index'].values
Lewis_C = Lewis_T.loc[Lewis_T.index.intersection(ID_Lewis_C)]

ID_MetaHit_C = MetaHit.loc[MetaHit["diagnosis"] == 'nonIBD', 'index'].values
MetaHit_C = MetaHit_T.loc[MetaHit_T.index.intersection(ID_MetaHit_C)]

ID_MetaHit2_C = MetaHit2.loc[MetaHit2["Health Status"] == "nonIBD", 'index'].values 
MetaHit2_C = MetaHit2_T.loc[MetaHit2_T.index.intersection(ID_MetaHit2_C)]

In [17]:
## List of DataFrame names
df_list = [("Eran Elinav" ,Eran_Elinav_C), ("IBDMDB", ibdmdb_C), ("IBDMDB2", ibdmdb2_C), ("HMP1", HMP1_C), ("HMP2",HMP2_C),
          ("HMP2 pilot", HMP2_pilot_C),("HMP3" , HMP3_C), ("Lewis", Lewis_C),("MetaHit", MetaHit_C), ("MetaHit2", MetaHit2_C)]
# Combine the DataFrames with "Source" column
dfs_with_source = []
for name, df in df_list:
    df_copy = df.copy()
    df_copy['Source'] = name
    dfs_with_source.append(df_copy)

combine_control_group_with_source = pd.concat(dfs_with_source, axis=0)

# Extract the 'Source' column before transformations
source_column = combine_control_group_with_source['Source']

# Apply transformations
df_numeric = combine_control_group_with_source.drop('Source', axis=1).apply(pd.to_numeric, errors='coerce', downcast='float')
df_numeric = df_numeric.loc[:, df_numeric.columns.str.startswith('s__')]
df_numeric = df_numeric.fillna(0)
df_numeric_filtered = df_numeric.loc[:, (df_numeric != 0).any(axis=0)]

# Reintroduce the 'Source' column
df_numeric_filtered['Source'] = source_column
combine_control_group = df_numeric_filtered

In [26]:
def n_samples_2_model (df_for_model):
    Feature = df_for_model.iloc[:,:len(df_for_model.columns)-1]
    Target = df_for_model.iloc[:,-1]
    X_train, X_test, y_train, y_test = train_test_split(Feature, Target, test_size=0.3, random_state=42,stratify=df_for_model[["Source"]])

    N_train_test_split = pd.DataFrame(y_train.value_counts())
    N_train_test_split.rename({'Type':'n_y_train'},axis=1, inplace=True)
    N_train_test_split['n_y_test'] = y_test.value_counts()
    n_classes = len(N_train_test_split.index)
    display(N_train_test_split)
    return X_train, X_test, y_train, y_test, n_classes

In [31]:
def RF_for_10(X_train, X_test, y_train, y_test):

    y_score = []
    Accuracy_10 = pd.DataFrame(columns=['Accuracy', 'Runtime'])
    iter = 1

    while iter <= 10:

        start_time = time.time()
        n_estimators_RF = [50, 100, 250, 500, 1000]
        max_features_RF = [2, 4, 6, 8, 10, 12, 14, None]
        parameters_RF = {'n_estimators': n_estimators_RF, 'max_features': max_features_RF}

        RF_model = RandomForestClassifier(max_depth=None, min_samples_split=2)
        RF_clf = GridSearchCV(RF_model, parameters_RF, cv=5, scoring="roc_auc")

        RF_clf.fit(X_train, y_train)

        print("Best Model:")
        RF_model = RF_clf.best_estimator_
        print(RF_model)

        RF_predictions = RF_model.predict(X_test)
        
        # creating a confusion matrix
        labels = np.unique(y_test)
        cm_RF = confusion_matrix(y_test, RF_predictions, labels=labels)
        cm_RF = pd.DataFrame(cm_RF, index=labels, columns=labels)
        sns.heatmap(cm_RF, cmap="YlGnBu", annot=True)
        
        Accuracy = round(metrics.accuracy_score(y_test, RF_predictions) * 100, 2)
        Accuracy_2_time = int(time.time() - start_time)
        
        # Append the results to the DataFrame
        temp_df = pd.DataFrame({'Accuracy': [Accuracy], 'Runtime': [Accuracy_2_time]})
        Accuracy_10 = Accuracy_10.append(temp_df, ignore_index=True)

        y_score.append(RF_clf.predict_proba(X_test))
        plt.show()
        
        iter += 1

    Accuracy_10.loc['mean'] = Accuracy_10.mean()
    Accuracy_10.loc['sd'] = Accuracy_10.std()
    y_score_10 = np.mean(y_score, axis=0)
    
    display(Accuracy_10)
    
    return Accuracy_10, y_score_10, y_test


In [28]:
def plot_multiclass_roc(y_score, y_test, n_classes, figsize=(12, 6)):

    # structures
    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    # calculate dummies once
    y_test_dummies = pd.get_dummies(y_test, drop_first=False).values
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test_dummies[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # roc for each class
    fig, ax = plt.subplots(figsize=figsize)
    ax.plot([0, 1], [0, 1], 'k--')
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title('Receiver operating characteristic for Optimized RF model')
    labels = np.unique(y_test)
    for i in range(n_classes):
        ax.plot(fpr[i], tpr[i], label=f'ROC curve (area = {round(roc_auc[i],2)}) for {labels[i]}')
    ax.legend(loc="best")
    ax.grid(alpha=.8)
    sns.despine()
    plt.show()
    return fig

without threshold

In [ ]:
X_train, X_test, y_train, y_test, n_classes = n_samples_2_model(combine_control_group)
Accuracy_10, y_score_10, y_test  = RF_for_10(X_train, X_test, y_train, y_test)
RF_model_optimized_roc_auc_curve = plot_multiclass_roc(y_score_10, y_test, n_classes=n_classes, figsize=(16, 8))

,count,n_y_test
Source,,
MetaHit2,132,57
HMP1,98,42
MetaHit,69,30
HMP2,66,28
HMP3,62,27
IBDMDB2,18,8
Lewis,18,8
IBDMDB,17,7
HMP2 pilot,17,7


Best Model:
RandomForestClassifier(max_features=2, n_estimators=50)


AttributeError: 'DataFrame' object has no attribute 'append'

Abundance > 0.1%

In [ ]:
control_trash = combine_control_group.copy()
control_R_Type = control_trash.loc[:,control_trash.columns != 'Source']
control_01 = control_R_Type[control_R_Type >= 0.1]
control_01['Source'] = control_trash['Source']
# Remove empty columns
control_01 = control_01.fillna(0)
all_zero = (control_01 == 0).all()
control_01 = control_01.loc[:,~all_zero]
control_01

In [ ]:
X_train, X_test, y_train, y_test, n_classes = n_samples_2_model(control_01)
Accuracy_10, y_score_10, y_test  = RF_for_10(X_train, X_test, y_train, y_test)
RF_model_optimized_roc_auc_curve = plot_multiclass_roc(y_score_10, y_test, n_classes=n_classes, figsize=(16, 8))

Abundance > 0.1% and column that are <= 80% empty

In [ ]:
control_trash = combine_control_group.copy()
control_R_Type = control_trash.loc[:,control_trash.columns != 'Source']
control_01_08 = control_R_Type[control_R_Type >= 0.1]
control_01_08 =control_01_08.loc[:,control_01_08.isnull().mean() <= .8]
control_01_08['Source'] = control_trash['Source']
# Remove empty columns
control_01_08 = control_01_08.fillna(0)
all_zero = (control_01_08 == 0).all()
control_01_08 = control_01_08.loc[:,~all_zero]
control_01_08

In [ ]:
X_train, X_test, y_train, y_test, n_classes = n_samples_2_model(control_01_08)
Accuracy_10, y_score_10, y_test  = RF_for_10(X_train, X_test, y_train, y_test)
RF_model_optimized_roc_auc_curve = plot_multiclass_roc(y_score_10, y_test, n_classes=n_classes, figsize=(16, 8))

Abundance > 0.1% and without MetaHit

In [ ]:
control_trash_01 = combine_control_group.copy()
control_R_Type = control_trash.loc[:,control_trash.columns != 'Source']
control_01 = control_R_Type[control_R_Type >= 0.1]
control_01['Source'] = control_trash['Source']
control_01 = control_01[control_01['Source'] != 'MetaHit']
control_01 = control_01.fillna(0)
all_zero = (control_01 == 0).all()
control_01_rm_M = control_01.loc[:,~all_zero]
control_01_rm_M

In [ ]:
X_train, X_test, y_train, y_test, n_classes = n_samples_2_model(control_01_rm_M)
Accuracy_10, y_score_10, y_test  = RF_for_10(X_train, X_test, y_train, y_test)
RF_model_optimized_roc_auc_curve = plot_multiclass_roc(y_score_10, y_test, n_classes=n_classes, figsize=(16, 8))

Abundance > 0.1% and without Lewis

In [ ]:
control_trash = combine_control_group.copy()
control_R_Type = control_trash.loc[:,control_trash.columns != 'Source']
control_01 = control_R_Type[control_R_Type >= 0.1]
control_01['Source'] = control_trash['Source']
control_01 = control_01[control_01['Source'] != 'Lewis']
control_01 = control_01.fillna(0)
all_zero = (control_01 == 0).all()
control_01_rm_L = control_01.loc[:,~all_zero]
control_01_rm_L

In [ ]:
X_train, X_test, y_train, y_test, n_classes = n_samples_2_model(control_01_rm_L)
Accuracy_10, y_score_10, y_test  = RF_for_10(X_train, X_test, y_train, y_test)
RF_model_optimized_roc_auc_curve = plot_multiclass_roc(y_score_10, y_test, n_classes=n_classes, figsize=(16, 8))

Abundance > 0.1% and without MetaHit and Lewis

In [ ]:
control_trash = combine_control_group.copy()
control_R_Type = control_trash.loc[:,control_trash.columns != 'Source']
control_01 = control_R_Type[control_R_Type >= 0.1]
control_01['Source'] = control_trash['Source']
control_01 = control_01[(control_01['Source'] != 'Lewis') & (control_01['Source'] != 'MetaHit')]
control_01 = control_01.fillna(0)
all_zero = (control_01 == 0).all()
control_01_rm_M_L = control_01.loc[:,~all_zero]
control_01_rm_M_L

In [ ]:
X_train, X_test, y_train, y_test, n_classes = n_samples_2_model(control_01_rm_M_L)
Accuracy_10, y_score_10, y_test  = RF_for_10(X_train, X_test, y_train, y_test)
RF_model_optimized_roc_auc_curve = plot_multiclass_roc(y_score_10, y_test, n_classes=n_classes, figsize=(16, 8))